## AutoEncoder Wind Farm C


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import os

# Custom EarlyStopping callback monitoring L2 norm reconstruction error on validation set
class L2NormEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, patience=3):
        super().__init__()
        self.X_val = validation_data[0]
        self.y_val = validation_data[1]  # same as X_val for AE
        self.patience = patience
        self.best_weights = None
        self.best_l2_norm = np.Inf
        self.wait = 0
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        val_pred = self.model.predict(self.X_val, verbose=0)
        l2_norm_error = np.linalg.norm(self.X_val - val_pred)
        print(f"Epoch {epoch+1}: Validation L2 norm reconstruction error = {l2_norm_error:.6f}")

        if l2_norm_error < self.best_l2_norm:
            self.best_l2_norm = l2_norm_error
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print(f"Early stopping triggered at epoch {self.stopped_epoch + 1}. Restoring best model weights.")
                self.model.set_weights(self.best_weights)

# Dataset list for WindFarm C 
dataset_ids = [1,4,5,6,8,9,11,12,15,16,18,20,28,29,30,31,32,33,35,36,37,39,41,43,44,46,47,
               48,49,50,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,75,76,78,79,80,81,85,88,89,90,91,93,94]


save_dir = r"D:\Master Thesis New Data Set\AutoEncoder\Wind Farm C\Power"
os.makedirs(save_dir, exist_ok=True)

for dataset_id in dataset_ids:
    print(f"\n=== Processing WindFarm C Dataset {dataset_id} ===")

    df = pd.read_csv(
        rf"D:\Master Thesis New Data Set\Final DataSet\Wind Farm C Final\{dataset_id}_WindFarm_C.csv",
        delimiter=','
    )

    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
    df['train_test'] = df['train_test'].astype(str).str.strip().str.lower()

    df['power_6_avg'] = df['power_6_avg'].apply(lambda x: max(x, 0))

        # Filter
    df_filtered = df[
        (df['sensor_76_avg'] <= 2) & 
        (df['sensor_76_avg'] >= -5) & 
        (df['sensor_124_avg'] >= -2) & 
        (df['sensor_124_avg'] <= 2) & 
        (df['status_type_id'] == 0) &
        (df['train_test'] == 'train')
    ].dropna(subset=['wind_speed_236_avg', 'power_6_avg'])

    df_sorted = df_filtered.sort_values(by='wind_speed_236_avg')
    original_max_power = df_sorted['power_6_avg'].max()
    rated_power = original_max_power
    power_threshold = 0.02 * rated_power
    print(f"Rated power: {rated_power:.4f}, Using power threshold: {power_threshold:.4f} (2% of rated power)")

    normal_df = df[
        (df['status_type_id'] == 0) &
        (df['train_test'] == 'train') &
        ~((df['wind_speed_236_avg'] >= 3) & 
          (df['wind_speed_236_avg'] <= 25) & 
          (df['power_6_avg'] <= power_threshold))
    ].copy()

    exclude_cols = ['time_stamp', 'train_test', 'id', 'asset_id']
    feature_cols = [c for c in df.columns if c not in exclude_cols]

    X = normal_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    if X.shape[0] < 50:
        print(f"⚠️ Skipping dataset {dataset_id} due to insufficient training data after filtering.")
        continue

    X_train, X_val = train_test_split(X, test_size=0.25, random_state=42)

    input_dim = X_train.shape[1]

    autoencoder = Sequential([
        Dense(133, activation='relu', input_shape=(input_dim,)),
        Dense(83, activation='relu'),
        Dense(20, activation='relu'),
        Dense(83, activation='relu'),
        Dense(133, activation='relu'),
        Dense(input_dim, activation='linear')
    ])
    autoencoder.compile(optimizer=Adam(learning_rate=0.0056), loss='mse')

    l2_early_stop = L2NormEarlyStopping(validation_data=(X_val, X_val), patience=3)

    autoencoder.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=200,
        batch_size=64,
        callbacks=[l2_early_stop],
        verbose=1
    )

    reconstructions_val = autoencoder.predict(X_val)
    reconstruction_errors_val = np.linalg.norm(X_val - reconstructions_val, axis=1)

    val_labels = normal_df.iloc[X_val.shape[0]*-1:]['status_type_id'].values
    val_labels = (val_labels != 0).astype(int)

    best_threshold = 0
    best_f1 = 0
    for threshold in np.linspace(min(reconstruction_errors_val), max(reconstruction_errors_val), 100):
        preds = (reconstruction_errors_val > threshold).astype(int)
        f1 = f1_score(val_labels, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print(f"📏 Selected threshold for WindFarm C Dataset {dataset_id}: {best_threshold:.6f} (F1 = {best_f1:.4f})")

    predict_df = df[df['train_test'] == 'prediction'].copy()
    if predict_df.empty:
        print(f"⚠️ No prediction data for dataset {dataset_id}, skipping prediction.")
        continue

    X_predict = predict_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    reconstructions_pred = autoencoder.predict(X_predict)
    actual_errors = np.linalg.norm(X_predict - reconstructions_pred, axis=1)

    is_anomaly = (actual_errors > best_threshold).astype(int)
    predict_df['predicted_status_type_id'] = np.where(is_anomaly == 1, 1, 0)

    save_path = os.path.join(save_dir, f"{dataset_id}_WindFarmC_status_predictions.csv")
    predict_df.to_csv(save_path, index=False)

    print(f"✅ Saved WindFarm C prediction for Dataset {dataset_id} — Anomalies detected: {sum(is_anomaly)}")

print("\n🎉 All WindFarm C datasets processed. AE-NBM predictions complete.")



=== Processing WindFarm C Dataset 61 ===
Rated power: 1.0223, Using power threshold: 0.0102 (10% of rated power)


C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
433/434 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 604737.5625Epoch 1: Validation L2 norm reconstruction error = 382102.440613
434/434 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 602691.1250 - val_loss: 16551.7344
Epoch 2/200
429/434 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 16725.9453Epoch 2: Validation L2 norm reconstruction error = 169519.964814
434/434 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 16677.0273 - val_loss: 3257.8071
Epoch 3/200
426/434 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3399.1936Epoch 3: Validation L2 norm reconstruction error = 158172.465975
434/434 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 3411.6838 - val_loss: 2836.2561
Epoch 4/200
432/434 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4133.2104Epoch 4: Validation L2 norm reconstruction error = 337866.462188
434/434 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 4133.8599 - val_loss: 12941.1777
Epoch 5/200
428/434 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5937.4688Epoch 5: Validation L2 norm reconstruction error = 157287.

C:\Users\49157\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


📏 Selected threshold for WindFarm C Dataset 61: 0.000000 (F1 = 0.0000)
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
✅ Saved WindFarm C prediction for Dataset 61 — Anomalies detected: 3025

=== Processing WindFarm C Dataset 62 ===
Rated power: 1.0238, Using power threshold: 0.0102 (10% of rated power)


...

📏 Selected threshold for WindFarm C Dataset 93: 0.000000 (F1 = 0.0000)
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
✅ Saved WindFarm C prediction for Dataset 93 — Anomalies detected: 3313

=== Processing WindFarm C Dataset 94 ===
Rated power: 1.0318, Using power threshold: 0.0103 (10% of rated power)


C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
393/396 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 736020.8125Epoch 1: Validation L2 norm reconstruction error = 720500.449209
396/396 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 730822.1875 - val_loss: 64525.3438
Epoch 2/200
391/396 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 49688.0664Epoch 2: Validation L2 norm reconstruction error = 458458.426910
396/396 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 49482.8672 - val_loss: 26125.3262
Epoch 3/200
396/396 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 131600.1406Epoch 3: Validation L2 norm reconstruction error = 461551.891683
396/396 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 131437.0156 - val_loss: 26479.0723
Epoch 4/200
392/396 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 25194.3359Epoch 4: Validation L2 norm reconstruction error = 478437.765052
396/396 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 25191.1914 - val_loss: 28451.9902
Epoch 5/200
393/396 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31001.1367Epoch 5: Validation L2 norm reconstruction error 

C:\Users\49157\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
✅ Saved WindFarm C prediction for Dataset 94 — Anomalies detected: 2737

🎉 All WindFarm C datasets processed. AE-NBM predictions complete.
